# Calculate and save seasonal averages of MetROMS ocean output

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import cmocean as cmo
import matplotlib.pyplot as plt
import subprocess

import matplotlib.gridspec as gridspec

In [2]:
# model info
diri      = '/g/data/gh9/wgh581/ROMS_Output/metroms_files/RAW/'
cntr_expt = 'metroms_CONTROL' 
pert_expt = 'metroms_4SSFLUX'
pert_2SSFLUX_expt = 'metroms_2SSFLUX'

# variable names
#varname   = ['dvidtt','dvidtd']
#varname   = ['aice','hi']
varname   = ['uvel','vvel']

#time domain for each experimnt
yrst = 2002 ; mst = 2
yren = 2011 ; men = 11 

#spatial domain (self-explanatory)
latmax = -50.
lonmin = 140#150.
lonmax = 300#300. 

# output file destination
path_to_data = '/g/data/gh9/wgh581/ROMS_Output/metroms_files/POST_PROCESS/'

#### Function to read data for each experiment

In [9]:
def test(ename):

    yrstr = [str(int) for int in np.arange(yrst, 1+yren)] 
    CMD  = 'ls '+diri+ename+'/cice/history/iceh.{'+','.join(yrstr)+'}-??-??.nc'
    print(CMD)

In [14]:
tets = test(cntr_expt)

ls /g/data/gh9/wgh581/ROMS_Output/metroms_files/RAW/metroms_CONTROL/cice/history/iceh.{2002,2003,2004,2005,2006,2007,2008,2009,2010,2011}-??-??.nc


In [10]:
def get_data(ename):
    
    # yrstr = [str(int) for int in np.arange(yrst, 1+yren)] 
    CMD  = 'ls '+diri+ename+'/ocean_avg_00??.nc'
    fili = subprocess.run(CMD, shell=True, capture_output=True)  #run CMD as a bash command
    fili = fili.stdout.decode().split('\n')[0:-1] 

    data = xr.open_mfdataset(fili, parallel=True)
    var  = data[varname]
   
    # reduce to specified spatial domain
    #var = var.where((data.TLAT <= latmax) & (data.TLON >= lonmin) & (data.TLON <= lonmax), drop=True)

    # Select surface value
    var = var.sel(s_rho=0, method='nearest')

    # get required time domain (time mean of)
    stdat = str(yrst)+'-'+str(mst)
    endat = str(yren)+'-'+str(men)
    var   = var.sel(ocean_time=slice(stdat,endat))#.mean('time')
    # rename time coordinate
    var   = var.rename({'ocean_time': 'time'})
               
    return var

Ocean surface velocities

In [52]:
%%time
varname = ['u', 'v']
velocity_cntr = get_data(cntr_expt).load()
velocity_cntr.to_netcdf(path_to_data + 'ocean_u_v_CONTROL_2002_2011_full_circumpolar.nc')

In [55]:
%%time
varname = ['u', 'v']
velocity_pert_2 = get_data(pert_2SSFLUX_expt).load()
velocity_pert_2.to_netcdf(path_to_data + 'ocean_u_v_2SSFLUX_2002_2011_full_circumpolar.nc')

CPU times: user 17 s, sys: 11.1 s, total: 28.1 s
Wall time: 38.5 s


In [56]:
%%time
varname = ['u', 'v']
velocity_pert_4 = get_data(pert_expt).load()
velocity_pert_4.to_netcdf(path_to_data + 'ocean_u_v_4SSFLUX_2002_2011_full_circumpolar.nc')

CPU times: user 16 s, sys: 11.1 s, total: 27.1 s
Wall time: 37 s


Ocean surface temperature

In [51]:
%%time
varname  = ['temp']
sst_cntr = get_data(cntr_expt).load()
sst_cntr.to_netcdf(path_to_data + 'ocean_SST_CONTROL_2002_2011_full_circumpolar.nc')

CPU times: user 15.3 s, sys: 8.48 s, total: 23.8 s
Wall time: 17.5 s


In [53]:
%%time
varname  = ['temp']
sst_pert_2 = get_data(pert_2SSFLUX_expt).load()
sst_pert_2.to_netcdf(path_to_data + 'ocean_SST_2SSFLUX_2002_2011_full_circumpolar.nc')

CPU times: user 14.1 s, sys: 8.73 s, total: 22.9 s
Wall time: 35.2 s


In [54]:
%%time
varname  = ['temp']
sst_pert_4 = get_data(pert_expt).load()
sst_pert_4.to_netcdf(path_to_data + 'ocean_SST_4SSFLUX_2002_2011_full_circumpolar.nc')

CPU times: user 13.9 s, sys: 8.72 s, total: 22.6 s
Wall time: 34 s


## 3d fields

In [3]:
yrst = 2008 ; mst = 12

In [4]:
def get_data_3d(ename, varname):
    
    # yrstr = [str(int) for int in np.arange(yrst, 1+yren)] 
    CMD  = 'ls '+diri+ename+'/ocean_avg_00??.nc'
    fili = subprocess.run(CMD, shell=True, capture_output=True)  #run CMD as a bash command
    fili = fili.stdout.decode().split('\n')[0:-1] 

    data = xr.open_mfdataset(fili, parallel=True)
    var  = data[varname]
   
    # reduce to specified spatial domain
    #var = var.where((data.TLAT <= latmax) & (data.TLON >= lonmin) & (data.TLON <= lonmax), drop=True)

    # Select surface value
    # var = var.sel(s_rho=0, method='nearest')

    # get required time domain (time mean of)
    stdat = str(yrst)+'-'+str(mst)
    endat = str(yren)+'-'+str(men)
    var   = var.sel(ocean_time=slice(stdat,endat))#.mean('time')
    # rename time coordinate
    var   = var.rename({'ocean_time': 'time'})
               
    return var

Control

In [5]:
%%time
varname  = ['temp']
temp_cntr = get_data_3d(cntr_expt, varname).load()
temp_cntr.to_netcdf(path_to_data + 'ocean_temp_3d_CONTROL_2008_2011_full_circumpolar.nc')

CPU times: user 33.6 s, sys: 34.6 s, total: 1min 8s
Wall time: 2min 42s


In [13]:
%%time
varname  = ['salt']
salt_cntr = get_data_3d(cntr_expt, varname).load()
salt_cntr.to_netcdf(path_to_data + 'ocean_salt_3d_CONTROL_2008_2011_full_circumpolar.nc')

CPU times: user 20.4 s, sys: 23.2 s, total: 43.6 s
Wall time: 52.2 s


2SSFLUX

In [5]:
%%time
varname  = ['temp']
temp_2SSFLUX = get_data_3d(pert_2SSFLUX_expt, varname).load()
temp_2SSFLUX.to_netcdf(path_to_data + 'ocean_temp_3d_2SSFLUX_2008_2011_full_circumpolar.nc')

CPU times: user 32.7 s, sys: 31.2 s, total: 1min 3s
Wall time: 1min 25s


In [6]:
%%time
varname  = ['salt']
salt_2SSFLUX = get_data_3d(pert_2SSFLUX_expt, varname).load()
salt_2SSFLUX.to_netcdf(path_to_data + 'ocean_salt_3d_2SSFLUX_2008_2011_full_circumpolar.nc')

CPU times: user 20.5 s, sys: 22.2 s, total: 42.7 s
Wall time: 44.9 s
